In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings 
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI 
from langchain.chains import ConversationalRetrievalChain


In [14]:
# file_paths = [
#     "/Users/boruchen/Documents/langchain-practice/ncku_CoM_data.csv",
#     "/Users/boruchen/Documents/langchain-practice/ncku_hub_data.csv"
# ]

# all_documents = []

# for file_path in file_paths:
#     loader = CSVLoader(file_path=file_path)
#     documents = loader.load()
#     all_documents.extend(documents)  

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# docs = text_splitter.split_documents(all_documents)

embeddings_model = OpenAIEmbeddings(openai_api_key='key')

# vectorstore = FAISS.from_documents(all_documents, embeddings_model)

# vectorstore.save_local("faiss_index")



new_db = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)


llm_model = ChatOpenAI(openai_api_key='key',temperature=0.7) 

# chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever(search_kwargs={"k": 3, "fetch_k": 10}, search_type="mmr"))
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever())



In [29]:
# query = "請根據資料,列出甜度為6的課程" 
query = "請列出會計系 : 初級會計學（一）的 grading"
response = chain.invoke({"question": query, "chat_history": []}) 
print(response) 
print()

query2 = "請列出會計系 : 初級會計學（一）出席分數佔比"
response2 = chain.invoke({"question": query2, "chat_history": []}) 
print(response2) 
print()

query3 = "會計系有什麼課程"
response3 = chain.invoke({"question": query3, "chat_history": []}) 
print(response3)
print()


{'question': '請列出會計系 : 初級會計學（一）的 grading', 'chat_history': [], 'answer': '期中考 Midterm Exam\t30\n期末考 Term exam\t40\n出席 Participation\t10\n作業 Assignments\t20'}

{'question': '請列出會計系 : 初級會計學（一）出席分數佔比', 'chat_history': [], 'answer': '初級會計學（一）出席佔總成績的10%。'}

{'question': '會計系有什麼課程', 'chat_history': [], 'answer': '會計系的課程如下：\n1. 公司治理與財務報導—國際會計與稅務專題研究\n2. 高級會計學（二）\n3. 公司理財\n\n這些課程涵蓋了公司治理、財務報導、會計審計、稅務等相關領域。'}



In [23]:
# 定義你的提示模板
prompt = """
課程背景：所有系所所開設課程中，根據不同科目以及開課老師，會有不同的總分算法。
問題：工資管系統計軟體與繪圖這門課中出席分數佔比多少？
AI:佔比為10%
"""

In [54]:
# 定義你的提示模板
prompt = """
背景信息：在這個系統中，各個系所開設的課程都有不同的評分標準。根據科目和任課老師，課程的總分構成會有所不同。並且你是一位專業的課程推薦AI負責精準的回答同學的問題。以下是一些示例問題及答案：

User:
問題：工資管理系統中的「統計軟體與繪圖」這門課，出席分數佔比多少？
AI：佔比為10%。


User：電子工程系「信號處理」課程的期末考試佔比多少？
AI：佔比為40%。


User：商業管理系「市場營銷」課程的平時作業佔比多少？
AI：佔比為30%。

User: 工資管系計算機該論這門課中出席分數佔比多少？
AI: 因為在我的資料庫中工資管系計算機該論這門課的課程大綱中並沒有描述出席分數的比重因此我無法回答你

User: 生物統計這門課中出席分數佔比多少？
課程背景：生物統計這門課中不包含出席分數
AI: 這門課程中出席分數並不會影響總分

User: 我想修有關機器學習的課程，請推薦一些適合我的課程?
AI: 統計系所開設的的機器學習以及敏求智慧學院開設的的人工智慧導論都是有關機器學習的課程

User: 工資系有什麼課程?
AI: 工資系開設以下課程：統計學、計算機概論、資料結構等


[指示]：根據上述背景信息和示例，回答以下問題：
User：{你的問題}
AI：

"""

In [55]:
# 定義你的提示模板
prompt = """The following are excerpts from conversations with an AI
assistant. The assistant is typically concise , producing
accurate responses to the users questions. Here are some
examples: 

User: 工資管系統計軟體與繪圖這門課中出席分數佔比多少？
AI: 工資管系統計軟體與繪圖這門課中出席分數佔比為20%

User: 工資管系計算機該論這門課中出席分數佔比多少？
AI: 因為在我的資料庫中工資管系計算機該論這門課的課程大綱中並沒有描述出席分數的比重因此我無法回答你

User: 生物統計這門課中出席分數佔比多少？
課程背景：生物統計這門課中不包含出席分數
AI: 這門課程中出席分數並不會影響總分

User: 我想修有關機器學習的課程
AI: 統計系所開設的的機器學習以及敏求智慧學院開設的的人工智慧導論都是有關機器學習的課程符合你的需求

User: 工資系有什麼課程?
AI: 工資系開設以下課程：統計學、計算機概論、資料結構等

User: 保險數學這門課中出席分數佔比多少？
AI: """
#我想修有關人工智慧的課程，請推薦我哪些是適合的
llm_model = ChatOpenAI(openai_api_key='key',temperature=0.7) 
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever())

# 設置查詢和 prompt
query = "有關人工智慧的課"
prompt = "請根據資料列出，" + query

# 調用鏈並顯示回應
response = chain.invoke({"question": query, "prompt": prompt, "chat_history": []})
print(response)



{'question': '有關人工智慧的課', 'prompt': '請根據資料列出，有關人工智慧的課', 'chat_history': [], 'answer': '請問您需要哪方面的資訊呢？這裡有幾門與人工智慧相關的課程提供的相關資訊，包括課程名稱、分數配比、開課系所、教學策略、課程大綱和課程描述。如果您有特定的問題或需要更多細節，請告訴我，我會盡量幫助您。'}


In [56]:
query = "這門課適合怎麼樣的人修？"
prompt = "請根據資料列出，" + query
#沒有記錄歷史詢問的功能剛剛才問過人工智慧卻不記得
# 調用鏈並顯示回應
response = chain.invoke({"question": query, "prompt": prompt, "chat_history": []})
print(response)

{'question': '這門課適合怎麼樣的人修？', 'prompt': '請根據資料列出，這門課適合怎麼樣的人修？', 'chat_history': [], 'answer': '這門課 "服務學習（一）" 適合那些有興趣參與校園認養區域的清潔與美化工作，以及對系所週邊和校園環境改善有興趣的人修讀。'}


In [53]:
llm_model = ChatOpenAI(openai_api_key='key') 
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever())

# 設置查詢和 prompt
query = "想學習有關人工智慧的課程"
prompt = "請根據資料列出，" + query

# 調用鏈並顯示回應
response = chain.invoke({"question": query, "prompt": prompt, "chat_history": []})
print(response)

{'question': '想學習有關人工智慧的課程', 'prompt': '請根據資料列出，想學習有關人工智慧的課程', 'chat_history': [], 'answer': '您可以考慮修習"人工智慧導論"這門課程。這門課程涵蓋了人工智慧的基本概念和技術，並強調使用Java或Python等程式語言開發智能應用。此外，課程中的作業、期中考、個人書面報告和出席也將有助於您學習和加深對人工智慧的理解。'}


In [47]:
query = "請根據資料,列出甜度超過6的課程" 
response = chain.invoke({"question": query, "chat_history": []}) 
print(response)

{'question': '請根據資料,列出甜度超過6的課程', 'chat_history': [], 'answer': '甜度超過6的課程有：\n- A9-304 應用化學與實驗\n- A9-305 應用化學與實驗\n- A9-114 應用統計'}


In [33]:
# query = "請根據資料,列出甜度為6的課程" 
query = "請列出會計系 : 初級會計學（一）的 grading"
response = chain.invoke({"question": query, "chat_history": []}) 
print(response) 
print()

query2 = "請列出會計系 : 初級會計學（一）出席分數佔比"
response2 = chain.invoke({ "question": query2,"chat_history": []}) 
print(response2) 
print()

query3 = "會計系有什麼課程"
response3 = chain.invoke({"question": query3, "chat_history": []}) 
print(response3)
print()


{'question': '請列出會計系 : 初級會計學（一）的 grading', 'chat_history': [], 'answer': '期中考 Midterm Exam: 30\n期末考 Term exam: 40\n出席 Participation: 10\n作業 Assignments: 20'}

{'question': '請列出會計系 : 初級會計學（一）出席分數佔比', 'chat_history': [], 'answer': '初級會計學（一）的出席分數佔比為10%。'}

{'question': '會計系有什麼課程', 'chat_history': [], 'answer': '會計系的課程包括：公司治理與財務報導—國際會計與稅務專題研、高級會計學（二）、公司理財等。'}

